In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
import random

rng = np.random.default_rng(seed=42)  # reproducible

In [2]:
# -----------------------------
# Helper functions
# -----------------------------
def random_datetimes(n, start, end, rng):
    """Generate n random datetimes between start and end (inclusive)."""
    start_ts = pd.Timestamp(start).value // 10**9
    end_ts = pd.Timestamp(end).value // 10**9
    ts = rng.integers(low=start_ts, high=end_ts + 1, size=n, endpoint=True)
    return pd.to_datetime(ts, unit="s")


In [3]:
# -----------------------------
# Table 5: dim_product (4 rows, labels)
# -----------------------------
product_categories = [
    ("Electronics", "ACME"),
    ("Furniture", "HomeCraft"),
    ("Clothing", "UrbanWear"),
    ("Grocery", "FreshFields"),
]
dim_product = pd.DataFrame(
    {
        "product_id": np.arange(1, 5, dtype=int),
        "product_category": [c for c, _ in product_categories],
        "brand": [b for _, b in product_categories],
        "is_perishable": [False, False, False, True],
    }
)

In [4]:
# Price anchors per category for fact rows
price_anchor = {
    1: 199.0,  # Electronics
    2: 350.0,  # Furniture
    3: 40.0,   # Clothing
    4: 5.0,    # Grocery
}

# -----------------------------
# Table 3: dim_customer (10 rows, labels)
# -----------------------------
segments = ["Consumer", "Corporate", "Enterprise", "SMB"]
regions = ["North", "South", "East", "West", "Central"]

dim_customer = pd.DataFrame(
    {
        "customer_id": np.arange(1, 11, dtype=int),
        "customer_name": [f"Customer {i}" for i in range(1, 11)],
        "segment": rng.choice(segments, size=10, replace=True),
        "region": rng.choice(regions, size=10, replace=True),
        "loyalty_tier": rng.choice(["Bronze", "Silver", "Gold", "Platinum"], size=10, replace=True),
    }
)

In [5]:
# -----------------------------
# Table 4: dim_store (50 rows, continuous)
# -----------------------------
cities = [
    "London", "Birmingham", "Manchester", "Leeds", "Glasgow",
    "Liverpool", "Bristol", "Sheffield", "Edinburgh", "Cardiff"
]
# Reasonable continuous fields: area (sqft), latitude, longitude, target_daily_sales
store_area = np.clip(rng.normal(loc=20000, scale=5000, size=50), 5000, 50000)
target_daily_sales = np.clip(rng.normal(loc=25000, scale=8000, size=50), 5000, 60000)

# Rough UK lat/lon bounds for realism (not exact)
latitudes = rng.uniform(50.0, 57.5, size=50)
longitudes = rng.uniform(-6.0, 1.5, size=50)

dim_store = pd.DataFrame(
    {
        "store_id": np.arange(1, 51, dtype=int),
        "store_name": [f"Store {i}" for i in range(1, 51)],
        "city": rng.choice(cities, size=50, replace=True),
        "store_area_sqft": np.round(store_area, 0).astype(int),
        "latitude": np.round(latitudes, 5),
        "longitude": np.round(longitudes, 5),
        "target_daily_sales": np.round(target_daily_sales, 2),
    }
)

In [6]:
# -----------------------------
# Table 2: dim_reading (1,000 rows, continuous)
# -----------------------------
# Think of these as IoT sensor readings; each is referenced by the fact table.
reading_times = random_datetimes(
    n=1000,
    start="2024-01-01 00:00:00",
    end="2025-10-24 23:59:59",
    rng=rng,
)

temperature_c = rng.normal(loc=20.0, scale=5.0, size=1000)         # ~20C with variation
humidity_pct = rng.uniform(20.0, 90.0, size=1000)                  # 20% - 90%
pressure_kpa = rng.normal(loc=101.3, scale=2.0, size=1000)         # around 101.3 kPa
vibration_rms = np.abs(rng.normal(loc=0.5, scale=0.2, size=1000))  # non-negative

dim_reading = pd.DataFrame(
    {
        "reading_id": np.arange(1, 1001, dtype=int),
        "reading_ts": reading_times.sort_values().values,
        "temperature_c": np.round(temperature_c, 2),
        "humidity_pct": np.round(humidity_pct, 2),
        "pressure_kpa": np.round(pressure_kpa, 2),
        "vibration_rms": np.round(vibration_rms, 3),
    }
)

In [7]:
# -----------------------------
# Table 1: fact_sales (main fact table, 1,000 rows)
# -----------------------------
# Foreign keys: customer_id -> dim_customer, product_id -> dim_product,
#               store_id -> dim_store, reading_id -> dim_reading
# We'll create a 1:1 mapping between fact rows and reading_id (to satisfy 1000 FK values),
# and sample other dimensions with realistic distributions.

# Transaction timestamps roughly correlate with the reading timestamp plus a small jitter.
reading_ids = np.arange(1, 1001, dtype=int)
rng.shuffle(reading_ids)


In [8]:
base_times = dim_reading.set_index("reading_id").loc[reading_ids, "reading_ts"].values
jitter_seconds = rng.integers(-3600, 3600, size=1000)  # +/- 1 hour
fact_timestamps = pd.to_datetime(base_times) + pd.to_timedelta(jitter_seconds, unit="s")

customer_ids = rng.integers(1, 11, size=1000)  # 1..10
product_ids = rng.integers(1, 5, size=1000)    # 1..4
store_ids = rng.integers(1, 51, size=1000)     # 1..50

# Quantity depends weakly on product category (e.g., groceries bought in multiples)
quantity = []
unit_price = []
for pid in product_ids:
    q = int(np.clip(rng.normal(loc=2.0 if pid == 4 else 1.2, scale=0.8, size=1)[0], 1, 8))
    anchor = price_anchor[pid]
    # Add category-specific noise
    price = np.round(float(np.clip(rng.normal(loc=anchor, scale=anchor * 0.15, size=1)[0], anchor*0.5, anchor*2.0)), 2)
    quantity.append(q)
    unit_price.append(price)

quantity = np.array(quantity, dtype=int)
unit_price = np.array(unit_price, dtype=float)
amount = np.round(quantity * unit_price, 2)

fact_sales = pd.DataFrame(
    {
        "sale_id": np.arange(1, 1001, dtype=int),
        "sale_ts": fact_timestamps,
        "customer_id": customer_ids,
        "product_id": product_ids,
        "store_id": store_ids,
        "reading_id": reading_ids,  # FK to dim_reading (1:1 here)
        "quantity": quantity,
        "unit_price": unit_price,
        "amount": amount,
    }
)

In [11]:
# -----------------------------
# Save to CSV
# -----------------------------
out_dir = Path(".")
out_dir.mkdir(parents=True, exist_ok=True)

paths = {
    "fact_sales": out_dir / "star_schema_fact_sales.csv",
    "dim_reading": out_dir / "star_schema_dim_reading.csv",
    "dim_customer": out_dir / "star_schema_dim_customer.csv",
    "dim_store": out_dir / "star_schema_dim_store.csv",
    "dim_product": out_dir / "star_schema_dim_product.csv",
}

fact_sales.to_csv(paths["fact_sales"], index=False)
dim_reading.to_csv(paths["dim_reading"], index=False)
dim_customer.to_csv(paths["dim_customer"], index=False)
dim_store.to_csv(paths["dim_store"], index=False)
dim_product.to_csv(paths["dim_product"], index=False)

In [12]:
# -----------------------------
# Basic integrity checks
# -----------------------------
# Ensure FK coverage
assert set(fact_sales["customer_id"].unique()).issubset(set(dim_customer["customer_id"]))
assert set(fact_sales["product_id"].unique()).issubset(set(dim_product["product_id"]))
assert set(fact_sales["store_id"].unique()).issubset(set(dim_store["store_id"]))
assert set(fact_sales["reading_id"].unique()).issubset(set(dim_reading["reading_id"]))


# Show concise schema summaries
def df_schema(df):
    return pd.DataFrame({"dtype": df.dtypes.astype(str), "non_null": df.notnull().sum(), "n_unique": df.nunique()})

schemas = {
    "fact_sales": df_schema(fact_sales),
    "dim_reading": df_schema(dim_reading),
    "dim_customer": df_schema(dim_customer),
    "dim_store": df_schema(dim_store),
    "dim_product": df_schema(dim_product),
}

schemas

{'fact_sales':                       dtype  non_null  n_unique
 sale_id               int64      1000      1000
 sale_ts      datetime64[ns]      1000      1000
 customer_id           int64      1000        10
 product_id            int64      1000         4
 store_id              int64      1000        50
 reading_id            int64      1000      1000
 quantity              int64      1000         5
 unit_price          float64      1000       901
 amount              float64      1000       950,
 'dim_reading':                         dtype  non_null  n_unique
 reading_id              int64      1000      1000
 reading_ts     datetime64[ns]      1000      1000
 temperature_c         float64      1000       780
 humidity_pct          float64      1000       941
 pressure_kpa          float64      1000       560
 vibration_rms         float64      1000       559,
 'dim_customer':                 dtype  non_null  n_unique
 customer_id     int64        10        10
 customer_name  obje